# For colab link: https://colab.research.google.com/drive/1H_y8XD3WuSgj0DXsxfVGGO-X53WbL9TR?usp=sharing
## list of skill used: Jupyter notebooks, Colab, Use of repos/models from huggingface or other public sources, Neural networks, GitHub, SQL (6 in total)

# install mysql package

In [1]:
%pip install mysql-connector-python


# set mysql function for data loading

In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from pandas import DataFrame, read_csv,read_sql_query

def create_connection():
    try:
        connection = mysql.connector.connect(host='dbnewyorkcartoon.cgyqzvdc98df.us-east-2.rds.amazonaws.com',
                                             database='new_york_cartoon',
                                             user='dbuser',
                                             password='Sql123456')
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            return connection
    except Error as e:
        print("Error while connecting to MySQL", e)
        return None

def select(q):
    connection = create_connection()
    if connection:
        try:
            df = read_sql_query(q, connection)
            return df
        except Error as e:
            print("Error executing the query:", e)
        finally:
            connection.close()
    return None

# select desired data from sql (cartoon #815) and store data in pandas dataframe

In [2]:
table = select("""
SELECT caption,mean
    FROM result
    WHERE contest_num=815

""")
table
df = pd.DataFrame(table)
df

Connected to MySQL Server version  8.0.33


<ipython-input-1-1053021abf91>:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = read_sql_query(q, connection)


,caption,mean
0,This place is known for serving local produce.,1.90778
1,Appears to be a food group .,1.90315
2,Elbows on the table. Another reason I don't l...,1.88494
3,"Well, you did tell the waiter to serve the veg...",1.85852
4,"Well, if I was going to spoil in a few days, I...",1.85081
...,...,...
6339,They look disgusting.,1.02222
6340,"Well, the menu did say they serve ‘meat and th...",1.02222
6341,"Grass fed and organic ,only the best for us v...",1.02128
6342,No wonder they're not vegitarian.,1.02041


# change data type to str for nlp

In [3]:
df["caption"] = df["caption"].astype(str)
df.dtypes


caption     object
mean       float64
dtype: object

# nlp

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('punkt')

#lowercase
df['caption'] = df['caption'].str.lower()
#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))
df['caption'] = df['caption'].apply(remove_punctuation)
df['caption'] = df['caption'].str.replace("‘", '')
df['caption'] = df['caption'].str.replace("’", '')
#tokenize
df['caption_edit'] = df['caption'].apply(lambda x: word_tokenize(x))

#remove stop words
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    words = text
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
df['caption_edit2'] = df['caption_edit'].apply(remove_stop_words)


#check
df



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,caption,mean,caption_edit,caption_edit2
0,this place is known for serving local produce,1.90778,"[place, known, serving, local, produce]",place known serving local produce
1,appears to be a food group,1.90315,"[appears, food, group]",appears food group
2,elbows on the table another reason i dont lik...,1.88494,"[elbows, table, another, reason, dont, like, b...",elbows table another reason dont like broccoli
3,well you did tell the waiter to serve the vege...,1.85852,"[well, tell, waiter, serve, vegetables, first]",well tell waiter serve vegetables first
4,well if i was going to spoil in a few days id ...,1.85081,"[well, going, spoil, days, id, steak]",well going spoil days id steak
...,...,...,...,...
6339,they look disgusting,1.02222,"[look, disgusting]",look disgusting
6340,well the menu did say they serve meat and three,1.02222,"[well, menu, say, serve, meat, three]",well menu say serve meat three
6341,grass fed and organic only the best for us ve...,1.02128,"[grass, fed, organic, best, us, veggies]",grass fed organic best us veggies
6342,no wonder theyre not vegitarian,1.02041,"[wonder, theyre, vegitarian]",wonder theyre vegitarian


# tensorflow train a model

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split

#split data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# Tokenize the text
max_words = 1000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['caption_edit2'])
train_sequences = tokenizer.texts_to_sequences(train_df['caption_edit2'])
test_sequences = tokenizer.texts_to_sequences(test_df['caption_edit2'])

# Pad sequences to a consistent length
max_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
train_padded_sequences = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Convert values to a numpy array
train_values = np.array(train_df['mean'], dtype=np.float64)
test_values = np.array(test_df['mean'], dtype=np.float64)

# Build a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Train the model
model.fit(train_padded_sequences, train_values, epochs=10, verbose=1, validation_data=(test_padded_sequences, test_values))




Epoch 1/10
159/159 [==============================] - 2s 5ms/step - loss: 0.4416 - mae: 0.5115 - val_loss: 0.0155 - val_mae: 0.0918
Epoch 2/10
159/159 [==============================] - 1s 4ms/step - loss: 0.0138 - mae: 0.0871 - val_loss: 0.0150 - val_mae: 0.0904
Epoch 3/10
159/159 [==============================] - 1s 5ms/step - loss: 0.0132 - mae: 0.0852 - val_loss: 0.0148 - val_mae: 0.0882
Epoch 4/10
159/159 [==============================] - 2s 10ms/step - loss: 0.0126 - mae: 0.0831 - val_loss: 0.0143 - val_mae: 0.0878
Epoch 5/10
159/159 [==============================] - 1s 5ms/step - loss: 0.0122 - mae: 0.0815 - val_loss: 0.0140 - val_mae: 0.0871
Epoch 6/10
159/159 [==============================] - 1s 3ms/step - loss: 0.0118 - mae: 0.0801 - val_loss: 0.0138 - val_mae: 0.0866
Epoch 7/10
159/159 [==============================] - 0s 3ms/step - loss: 0.0114 - mae: 0.0787 - val_loss: 0.0139 - val_mae: 0.0856
Epoch 8/10
159/159 [==============================] - 0s 2ms/step - loss: 0

# example prediction
Let's test the best caption with mean 1.9078.

In [18]:

sample_text = ["This place is known for serving local produce."]
sample_sequence = tokenizer.texts_to_sequences(sample_text)
sample_padded_sequence = pad_sequences(sample_sequence, maxlen=max_length, padding='post', truncating='post')
prediction = model.predict(sample_padded_sequence)
print("Predicted value:", prediction[0][0])

1/1 [==============================] - 0s 22ms/step
Predicted value: 1.3264782


# hugging face
## install transformers

In [24]:
%pip install transformers

## too many captions, for faster, pick up small amount of captions for the test

In [43]:
table = select("""
SELECT caption,mean
    FROM result
    WHERE contest_num=815 AND mean>1.5

""")
table
df2 = pd.DataFrame(table)
df2

Connected to MySQL Server version  8.0.33


<ipython-input-1-1053021abf91>:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = read_sql_query(q, connection)


,caption,mean
0,This place is known for serving local produce.,1.90778
1,Appears to be a food group .,1.90315
2,Elbows on the table. Another reason I don't l...,1.88494
3,"Well, you did tell the waiter to serve the veg...",1.85852
4,"Well, if I was going to spoil in a few days, I...",1.85081
...,...,...
242,Good luck getting these pronouns right.,1.50323
243,I don't know what they're steamed about!,1.50323
244,I'm just trying to figure out if Alanis Moriss...,1.50319
245,"I'm starting to think that ""100% plant-based"" ...",1.50296


## Test captions with sentiment analysis model from hugging face

In [45]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion')
sentiments = []
for text in df2['caption']:
    sentiment = classifier(text)
    sentiments.append(sentiment[0])
df2['sentiment'] = sentiments
df2

,caption,mean,sentiment
0,This place is known for serving local produce.,1.90778,"{'label': 'joy', 'score': 0.8695807456970215}"
1,Appears to be a food group .,1.90315,"{'label': 'anger', 'score': 0.8076922297477722}"
2,Elbows on the table. Another reason I don't l...,1.88494,"{'label': 'anger', 'score': 0.6099650263786316}"
3,"Well, you did tell the waiter to serve the veg...",1.85852,"{'label': 'joy', 'score': 0.6556003093719482}"
4,"Well, if I was going to spoil in a few days, I...",1.85081,"{'label': 'joy', 'score': 0.8716171979904175}"
...,...,...,...
242,Good luck getting these pronouns right.,1.50323,"{'label': 'joy', 'score': 0.992285966873169}"
243,I don't know what they're steamed about!,1.50323,"{'label': 'anger', 'score': 0.9930276274681091}"
244,I'm just trying to figure out if Alanis Moriss...,1.50319,"{'label': 'anger', 'score': 0.9608468413352966}"
245,"I'm starting to think that ""100% plant-based"" ...",1.50296,"{'label': 'joy', 'score': 0.9483328461647034}"
